# Fingerprint Tutorial 1

Updated: 04/12/2025

**Contributing Authors**  
Andrew Trepagnier,
*Mississippi State University*  
📧 andrew.trepagnier1@gmail.com

---

## Objective

To understand the process of training an interatomic potential, it is helpful to begin by demonstrating how structural fingerprints encode an atom’s local atomic environment as input to a neural network. This document is intended for anyone interested in training their own interatomic potential or learning more about the computational strategies used in Machine-Learned Interatomic Potentials (MLIPs).


## Introduction

Structural fingerprints are mathematical representations of an atom’s local atomic environment within a system of atoms or molecules. These fingerprints are crucial for converting high-dimensional data—such as the distances between atom i and atom j across hundreds of atoms and simulation timesteps—into lower-dimensional descriptors that still retain essential environmental information.

In this tutorial, I provide a step-by-step walkthrough for manually computing structural fingerprints, as seen in the referenced study. While the included fingerprint scripts can automatically parse input files and compute these descriptors efficiently, manually working through a basic example can help build intuition and strengthen understanding of how structural information is encoded.



<img src="pics/fingerprintconversion.jpeg" alt="Diagram" width="500">

<div style="background-color:#f1f8e9; padding: 20px; border-left: 6px solid #7cb342; border-radius: 10px; font-size: 16px; line-height: 1.6">
    
## Priors

The priors listed below outline the two encoded fingerprint expressions and their associated variables for reference. The computation walkthrough will show how these are used practically.

### 🔷 Structural Fingerprint Expressions

- <b>Pair Interaction Definition</b>  
  $$
  F_{(n-o)} = \sum_{j \neq i} \left( \frac{r_{ij}}{r_e} \right)^{(n-o)} e^{-\alpha_n \frac{r_{ij}}{r_e}} f_c \left( \frac{r_c - r_{ij}}{\Delta r} \right) 
  $$

- <b>3-Body Fingerprint</b>  
  $$
  \begin{split}
  G_{m,k} = \sum_{j,k} \cos^m \theta_{jik} \; & e^{-\beta_k \frac{r_{ij} + r_{ik}}{r_e}} 
  f_c \left( \frac{r_c - r_{ij}}{\Delta r} \right) \\
  & \times f_c \left( \frac{r_c - r_{ik}}{\Delta r} \right) 
  \end{split}
  $$


 <p> Where the Cutoff Function, $f_{c}$, is defined as</p>
    $$    
    f_c(x) =
    \begin{cases} 
    1, & x > 1 \\
    \left( 1 - (1 - x)^4 \right)^2, & 0 \leq x \leq 1 \\
    0, & x < 0
    \end{cases}
  $$

---

### 🔹 Pair-Interaction Variables

- \( n \): Integer that adjusts the number of fingerprints in the input layer  
- Euclidean distance between atom \( i \) and atom \( j \):  
  $$
  r_{ij} = \sqrt{(x_{j}-x_{i})^2 + (y_{j}-y_{i})^2 + (z_{j}-z_{i})^2}
  $$
- \( $r_e$ \): Equilibrium distance between atoms  
- \( $r_c$ \): Cutoff radius  
- \( $\Delta$r \): Difference in equilibrium radius and cutoff radius  
- \( $\alpha_n$ \): The \( n \)th alpha in the vector of alphas, based on the bulk modulus of the material  

---

### 🔹 3-Body Variables

- \( m \): Integer  
- \( k \): Integer  
- \( $\theta_{ijk}$ \): Interatomic angle between atom \( i \), atom \( j \), and atom \( k \)  
- Euclidean distance between atom \( i \) and atom \( j \):  
  $$
  r_{ij} = \sqrt{(x_{j}-x_{i})^2 + (y_{j}-y_{i})^2 + (z_{j}-z_{i})^2}
  $$
- \( $r_e$ \): Equilibrium distance between atoms  
- \( $r_c$ \): Cutoff radius  
- \( $\Delta$ r \): Difference in equilibrium radius and cutoff radius  
- \( $\beta_k$ \): Damping coefficient for distance penalty in 3-body term

</div>



## Step-by-Step Example

In [1]:
# Install Libraries
import numpy as np

We consider a simple 2D system of 4 atoms arranged in a square configuration. We could imagine the coordinate locations would look something like this:
$$
\begin{aligned}
{atom}_0 &= (0, 0) \\
{atom}_1 &= (2, 0) \\
{atom}_2 &= (0, 2) \\
{atom}_3 &= (2, 2)
\end{aligned}
$$


In [2]:
atom0 = [0, 0]
atom1 = [2, 0]
atom2 = [0, 2]
atom3 = [2, 2]

atom_positions = np.stack((atom0, atom1, atom2, atom3))
print(atom_positions)

[[0 0]
 [2 0]
 [0 2]
 [2 2]]


Visually, the system of 4 atoms would look like:

<img src="pics/box.jpeg" alt="Diagram" width="300">

Here, each colored node represents an atom in a 2D box. We consider pairwise distances and angles between atoms to compute a structural fingerprint vector for each atom.

### Distance Matrix

Next, we'll compute the distance matrix. For each ith atom that we consider, we want to calculate the euclidean distance between it and atom j, where atom j is each atom in the system (except i, itself, because that will always be zero). These are the ij distances. So for atom pair i=0 and j=1 (or also referred to as atom01), it would be computed as such:

$$ r_{01} = \sqrt{(2-0)^2 + (0-0)^2 } = 2$$

We need to do this for every pair of atoms, so let's make a nested for loop that would handle any nxm system of atom positions:


In [3]:
num_atoms = 4

distance_matrix = np.zeros([num_atoms, num_atoms])
for i in range(num_atoms):                                               
    for j in range(num_atoms):                                          
        if i != j:
            distance_matrix[i,j] = np.linalg.norm(atom_positions[i] - atom_positions[j])
print(distance_matrix)


[[0.         2.         2.         2.82842712]
 [2.         0.         2.82842712 2.        ]
 [2.         2.82842712 0.         2.        ]
 [2.82842712 2.         2.         0.        ]]


This can also be visualized as:

$$
\begin{array}{c|cccc}
        & \text{Atom}_0 & \text{Atom}_1 & \text{Atom}_2 & \text{Atom}_3 \\
\hline
\text{Atom}_0 & 0 & r_{01} & r_{02} & r_{03} \\
\text{Atom}_1 & r_{10} & 0 & r_{12} & r_{13} \\
\text{Atom}_2 & r_{20} & r_{21} & 0 & r_{23} \\
\text{Atom}_3 & r_{30} & r_{31} & r_{32} & 0 \\
\end{array}
$$

The size of the distance matrix grows exponentially with larger and larger systems(6x6 = 36 entries, 10x10= 100 entries, ...). Hence, why we must be strategic in how we compute the fingerprints with these values for systems of hundreds to thousands of atoms.


### Pair Interaction Fingerprint Computation


Here we will compute the pair interaction fingerprint. Below is an example of a list of metaparameters our input parsing function may extract:

> *Remember, these are not real values used in the study. They are to show how the fingerprint functions work.*

- `re = 1.1`
- `rc = 2.80`
- `αₙ = [6.95, 6.95, 6.95, 6.95]`
- `dr = 1.7`
- `o = -1`
- `n = 2`
- `n - o = 3`

According to our pair interaction fingerprint function, there will be **n - o** pair interaction fingerprints (array-based, so a total of **4**). Remember, our vector of fingerprints will be stacked together in a vector and act as the feature input like such:
Feature Input = 
$$
Feature Input = 
\begin{bmatrix}
F_{0} \\
F_{1} \\
F_{2} \\
F_{3} \\
F_{4} \\
G_{0,0}\\
. \\
. \\
.
\end{bmatrix}
$$





### Analytical interpretation of the Pair Interaction Fingerprints:

### For $F_{0}$:

<div style="background-color:#e0f7fa; padding: 15px; border-left: 5px solid #00796b; border-radius: 8px; margin-bottom: 10px;">
<b>Fingerprint Formula:</b><br>
$$
F_0 = \sum_{j \neq i} \left( \frac{r_{ij}}{r_e} \right)^0 \exp\left(-\alpha_0 \cdot \frac{r_{ij}}{r_e} \right) f_c\left( \frac{r_c - r_{ij}}{\Delta r} \right) S_{ij}
$$
</div>

<div style="display:flex; flex-direction:column; gap:10px;">

<!-- First term -->
<div style="background-color:#fff3e0; padding: 10px; border-left: 5px solid #fb8c00; border-radius: 5px;">
<b>Term 1 (j = 1):</b><br>
\( \left( \frac{r_{01}}{r_e} \right)^0 \cdot \exp\left(-\alpha_0 \cdot \frac{r_{01}}{r_e} \right) \cdot f_c\left( \frac{r_c - r_{01}}{\Delta r} \right) \cdot S_{01} \)
</div>

<div style="text-align:center; font-size: 20px;">+</div>

<!-- Second term -->
<div style="background-color:#fff3e0; padding: 10px; border-left: 5px solid #fb8c00; border-radius: 5px;">
<b>Term 2 (j = 2):</b><br>
\( \left( \frac{r_{02}}{r_e} \right)^0 \cdot \exp\left(-\alpha_0 \cdot \frac{r_{02}}{r_e} \right) \cdot f_c\left( \frac{r_c - r_{02}}{\Delta r} \right) \cdot S_{02} \)
</div>

<div style="text-align:center; font-size: 20px;">+</div>

<!-- Third term -->
<div style="background-color:#fff3e0; padding: 10px; border-left: 5px solid #fb8c00; border-radius: 5px;">
<b>Term 3 (j = 3):</b><br>
\( \left( \frac{r_{03}}{r_e} \right)^0 \cdot \exp\left(-\alpha_0 \cdot \frac{r_{03}}{r_e} \right) \cdot f_c\left( \frac{r_c - r_{03}}{\Delta r} \right) \cdot S_{03} \)
</div>

<div style="text-align:center; font-size: 20px;">+</div>

<!-- Ellipsis -->
<div style="text-align:center; font-size: 18px; color: #999;">
...and so on for all atoms \( j \ne i \)
</div>

</div>

---
### For $F_{1}$:

<div style="background-color:#e0f7fa; padding: 15px; border-left: 5px solid #00796b; border-radius: 8px; margin-bottom: 10px;">
<b>Fingerprint Formula:</b><br>
$$
F_1 = \sum_{j \neq i} \left( \frac{r_{ij}}{r_e} \right)^1 \exp\left(-\alpha_1 \cdot \frac{r_{ij}}{r_e} \right) f_c\left( \frac{r_c - r_{ij}}{\Delta r} \right) S_{ij}
$$
</div>

<div style="display:flex; flex-direction:column; gap:10px;">

<!-- First term -->
<div style="background-color:#fff3e0; padding: 10px; border-left: 5px solid #fb8c00; border-radius: 5px;">
<b>Term 1 (j = 1):</b><br>
\( \left( \frac{r_{01}}{r_e} \right)^1 \cdot \exp\left(-\alpha_1 \cdot \frac{r_{01}}{r_e} \right) \cdot f_c\left( \frac{r_c - r_{01}}{\Delta r} \right) \cdot S_{01} \)
</div>

<div style="text-align:center; font-size: 20px;">+</div>

<!-- Second term -->
<div style="background-color:#fff3e0; padding: 10px; border-left: 5px solid #fb8c00; border-radius: 5px;">
<b>Term 2 (j = 2):</b><br>
\( \left( \frac{r_{02}}{r_e} \right)^1 \cdot \exp\left(-\alpha_1 \cdot \frac{r_{02}}{r_e} \right) \cdot f_c\left( \frac{r_c - r_{02}}{\Delta r} \right) \cdot S_{02} \)
</div>

<div style="text-align:center; font-size: 20px;">+</div>

<!-- Third term -->
<div style="background-color:#fff3e0; padding: 10px; border-left: 5px solid #fb8c00; border-radius: 5px;">
<b>Term 3 (j = 3):</b><br>
\( \left( \frac{r_{03}}{r_e} \right)^1 \cdot \exp\left(-\alpha_1 \cdot \frac{r_{03}}{r_e} \right) \cdot f_c\left( \frac{r_c - r_{03}}{\Delta r} \right) \cdot S_{03} \)
</div>

<div style="text-align:center; font-size: 20px;">+</div>

<!-- Ellipsis -->
<div style="text-align:center; font-size: 18px; color: #999;">
...and so on for all atoms \( j \ne i \)
</div>

</div>


---
### Discussion

This would create n (array-based) pair interaction fingerprints (F0 ,F1 ,F2 ,…) that could be concatenated into an input feature vector. It is important to note that each atom will receive its own set of these fingerprints. So, if your fingerprint vector contains 29 entries, that would mean 29 entries per atom in the system.

Notice that i=0 remains the same in the computations above. The set of fingerprints described would be the feature input for atom i=0, since it encodes atom i's distance relationship with all other j atoms in the system.

Before computing the fingerprints of the 4-atom system, we should first create a lookup table of fingerprints. A lookup table is a precomputed table of values that can be used alongside interpolation techniques to quickly retrieve approximate values. This is done to enhance computational efficiency for the neural network. Instead of computing expensive fingerprint operations (e.g., exponentials, powers, cutoff functions, etc.) for thousands of timesteps, we can rapidly interpolate fingerprint values from a precomputed table.
In other words, we will generate a table of approximately 1000 radial distances ranging from 0 to rc , and compute the corresponding ~1000 fingerprint values. Any radial distances that fall between the precomputed values can then be interpolated using the Centripetal Catmull–Rom spline technique.

In [4]:
class Fingerprint_lookup_table:
    
    def __init__(self, atom_positions, num_atoms):
        
        self.atom_positions = atom_positions
        self.num_atoms = num_atoms
        self.re = 1.1
        self.rc = 2.0
        self.alphan = [6.95, 6.95, 6.95, 6.95] #We consider a system of a pure element in this example so bulk modulus remains the same, alloys are different
        self.dr = 1.7
        self.o = -1
        self.n = 2
        self.m = self.n - self.o

    def export_params(self):
        """
        Ignore this method unless you're exporting a dictionary of 
        the above listed parameters into the next class.
        """
        params = {
            "re": self.re,
            "rc": self.rc,
            "alphan": self.alphan,
            "dr": self.dr,
            "o": self.o,
            "n": self.n,
            "m": self.m
        }
        return params

    def cutoff(self, r) -> float:
        x = (self.rc - r) / self.dr
        if x > 1:
            return 1
        elif 0 <= x <= 1:
            return (1 - (1 - x)**4)**2 #Continuous cutoff range for smoother potentials
        else:
            return 0
        
    def radial_table(self) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
        buffer = 5
        res = 1000
    
        radii_table = np.zeros((res + buffer, int(self.n - self.o)), dtype=float)  # (1005, n-o)
        dfctable = np.zeros(res + buffer)
        r1 = np.zeros(res + buffer)
    
        for m in range(self.m):
            """
            Consider m=0 first, compute PI fingerprints for all m=0 and save in the 0th column of radii_table,
            then increment and fill in next column until all m's are computed and stored
            """
            for k in range(res + buffer):
                r1[k] = self.rc * k / res
                r1_sqrt = np.sqrt(r1[k])
        
                term_of_mth_fp = (
                    (r1_sqrt / self.re) ** m *
                    np.exp(-self.alphan[m] * (r1_sqrt / self.re)) *
                    self.cutoff(r1_sqrt)
                )
                radii_table[k, m] = term_of_mth_fp
        
                if r1_sqrt >= self.rc or r1_sqrt <= self.rc - self.dr:
                    dfctable[k] = 0
                else:
                    term = (self.rc - r1_sqrt) / self.dr
                    dfctable[k] = (-8 * (1 - term) ** 3) / (self.dr * (1 - term) ** 4)
    
        return r1, radii_table, dfctable

if __name__ == '__main__':
    fp = Fingerprint_lookup_table(atom_positions, num_atoms)
    r1 , radii_table, _ = fp.radial_table()
    params = fp.export_params()
    print(f"The radii_table: {radii_table}")
    print(f"The radii_table shape: {np.shape(radii_table)}")
    

The radii_table: [[1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.53853158e-01 3.06484892e-02 1.24603828e-03]
 [6.70590304e-01 3.85562316e-02 2.21682745e-03]
 ...
 [8.65763047e-05 1.11417964e-04 1.43387533e-04]
 [8.60913996e-05 1.10849196e-04 1.42726734e-04]
 [8.56092053e-05 1.10283270e-04 1.42068830e-04]]
The radii_table shape: (1005, 3)


We now have a precomputed lookup table where column 0 represents all m=0 fingerprint terms, column 1 represents all m=1 fingerprint terms, and column 2 represents all m=2 terms. In the particular example of a 4-atom system, a lookup table is overkill; however, if we scaled our atomic system up or evaluated millions of timesteps, then this would be necessary for computational efficiency.
Let's now interpolate the pair interaction fingerprints of our system of 4 atoms.

In [5]:
class Fingerprint: 
    
    def __init__(self, distance_matrix, r1, radii_table, atom_positions, num_atoms, params):
        self.distance_matrix = distance_matrix
        self.r1 = r1
        self.radii_table = radii_table
        self.atom_positions = atom_positions
        self.num_atoms = num_atoms
        self.params = params
        

    def compute_fingerprint(self) -> np.ndarray:
        
        num_fingerprints = int(self.params['m'])  
        fingerprints = np.zeros((self.num_atoms-1, num_fingerprints), dtype=float) # Initialize fingerprint array
        
        for i in range(self.num_atoms-1):
            for j in range(self.num_atoms-1):
                if i != j:  
                    rij = self.distance_matrix[i,j] #for every rij, we'll find its corresponding radius in the lookup table         
                    idx = np.searchsorted(r1, rij) - 1 
                    
                    if idx > 0 and idx < len(r1) - 2: # Ensure we have points for interpolation
                        t = (rij - r1[idx]) / (r1[idx+1] - r1[idx]) # Calculate interpolation parameter t
                        
                        for m in range(num_fingerprints):
                            y = [
                                radii_table[idx-1, m],
                                radii_table[idx, m],
                                radii_table[idx+1, m],
                                radii_table[idx+2, m]
                            ]

                            # Catmull-Rom interpolation
                            t2 = t * t
                            t3 = t2 * t
                            
                            # Catmull-Rom coefficients
                            p0 = -0.5*t3 + t2 - 0.5*t
                            p1 = 1.5*t3 - 2.5*t2 + 1.0
                            p2 = -1.5*t3 + 2.0*t2 + 0.5*t
                            p3 = 0.5*t3 - 0.5*t2
                            
                            # Interpolated value
                            interpolated_value = (
                                y[0] * p0 + 
                                y[1] * p1 + 
                                y[2] * p2 + 
                                y[3] * p3
                            )
                            
                            # Add contribution to fingerprint
                            fingerprints[i,m] += interpolated_value
        
        # Calculate sums after all atoms are processed
        summed_F0 = np.sum(fingerprints[:, 0])  # Sum of first fingerprint type
        summed_F1 = np.sum(fingerprints[:, 1])  # Sum of second fingerprint type
        summed_F2 = np.sum(fingerprints[:, 2])  # Sum of third fingerprint type
        self.fingerprints = fingerprints
        return summed_F0, summed_F1, summed_F2, fingerprints

In [6]:
fp = Fingerprint(distance_matrix, r1, radii_table, atom_positions, num_atoms, params)
F0, F1, F2, fingerprints = fp.compute_fingerprint()
print(f"The first input feature is F0: {F0}")
print("-------------------------------------")
print(f"The first input feature is F1: {F1}")
print("-------------------------------------")
print(f"The first input feature is F2: {F2}")
print("-------------------------------------")
print(f"The fingerprint array is: {fingerprints}")


The first input feature is F0: 0.00035021724113492236
-------------------------------------
The first input feature is F1: 0.00045025633835445075
-------------------------------------
The first input feature is F2: 0.000578871472950285
-------------------------------------
The fingerprint array is: [[1.75108621e-04 2.25128169e-04 2.89435736e-04]
 [8.75543103e-05 1.12564085e-04 1.44717868e-04]
 [8.75543103e-05 1.12564085e-04 1.44717868e-04]]
